In [1]:
import sqlite3
import pandas as pd
from soynlp.noun import LRNounExtractor_v2
from collections import defaultdict
import re
import numpy as np
import math
from string import punctuation

# Stopwords 처리
pattern1 = re.compile(r'[{}]'.format(re.escape(punctuation))) # punctuation 제거
pattern2 = re.compile(r'[^가-힣 ]') # 특수문자, 모음, 숫자, 영어 제거
pattern3 = re.compile(r'\s{2,}') # white space 1개로 바꾸기.

class Extracter:
    def __init__(self):
        self.noun_extractor = LRNounExtractor_v2(verbose=True)
        
    def cleaning(self,text):
        return pattern3.sub(' ', 
                  pattern2.sub('',
                   pattern1.sub('', text)))

    def extract_nouns(self,df):
        tempary = np.linspace(0,1,11)
        nouns = [self.noun_extractor.train_extract(_['head'], min_noun_frequency=50) for _ in [df.iloc[math.ceil(len(df)*tempary[_]):math.ceil(len(df)*tempary[_+1])] for _ in range(len(tempary)-1)]]
        words = {k:v for i in range(len(nouns)) for k,v in nouns[i].items() if len(k) > 1}
        return words

    def search_dict(self,nouns):
        # 사전 검색 결과 없는 단어 추출
        conn = sqlite3.connect('kr_korean.db')
        cur = conn.cursor()
        data = pd.read_sql('SELECT word FROM kr WHERE part="명사"', conn)
        data = ' '.join(data['word'])
        return pd.DataFrame([_ for _ in nouns if _[0] not in data])

In [2]:
conn = sqlite3.connect('Humor.db')
cur = conn.cursor()
df = pd.read_sql('SELECT head FROM head',conn)
df.drop_duplicates(keep='first')
ext = Extracter()
df['head'] = df['head'].map(lambda x:ext.cleaning(x))
nouns = ext.extract_nouns(df)
words = sorted(nouns.items(),key=lambda _:_[1], reverse=True)
new_words = ext.search_dict(words)

[Noun Extractor] use default predictors
[Noun Extractor] num features: pos=3929, neg=2321, common=107
[Noun Extractor] counting eojeols
[EojeolCounter] n eojeol = 141099 from 125252 sents. mem=0.485 Gb                    
[Noun Extractor] complete eojeol counter -> lr graph
[Noun Extractor] has been trained. #eojeols=532046, mem=0.680 Gb
[Noun Extractor] batch prediction was completed for 37314 words
[Noun Extractor] checked compounds. discovered 21766 compounds
[Noun Extractor] postprocessing detaching_features : 1249 -> 1244
[Noun Extractor] postprocessing ignore_features : 1244 -> 1190
[Noun Extractor] postprocessing ignore_NJ : 1190 -> 1182
[Noun Extractor] 1182 nouns (21766 compounds) with min frequency=50
[Noun Extractor] flushing was done. mem=0.747 Gb                    
[Noun Extractor] 44.91 % eojeols are covered
[Noun Extractor] counting eojeols
[EojeolCounter] n eojeol = 150363 from 125252 sents. mem=0.795 Gb                    
[Noun Extractor] complete eojeol counter -> l